## IMPORT RELEVANT LIBRARIES

In [1]:
import pyodbc

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## DATABASE CONNECTION

In [2]:
db_path = r"C:\Users\GILBERT FG\Desktop\Readings\PSP_database.accdb"

conn = pyodbc.connect(
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf"DBQ={db_path};"
)

cursor = conn.cursor()

# read all tables
for table in cursor.tables(tableType='TABLE'):
    print(table.table_name)

Block register
Plot monitoring history
Plot-block relation
PSPs
Slope correction factors
Thinning history
TreeData


## DATA EXTRACTION AND AMALGAMATION

In [3]:
query_blk = r"SELECT * FROM [Block register]"

blk_register = pd.read_sql(query_blk, conn)
blk_register.head()

C:\Users\GILBERT FG\AppData\Local\Temp\ipykernel_14700\1003375522.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  blk_register = pd.read_sql(query_blk, conn)


,PLANTATION,BLOCK,STATUS,ROTATION,OWNERSHIP,RESERVE,YEAR PLANTED,BLOCK AREA (ha),PRODUCTIVE,UNPRODUCTIVE,TEAK,NATURAL INDIGENOUS,PLANTED INDIGENOUS,GMELINA,OPEN LAND,ROCK,REMARK
0,Akumadan,2A,PLANTED,1,FORM GHANA,ASUBIMA,2008,77.853644,77.853644,0.00,77.853644,NaN,NaN,NaN,NaN,NaN,None
1,Akumadan,2B,PLANTED,1,FORM GHANA,ASUBIMA,2010,4.523337,4.523337,NaN,4.523337,NaN,NaN,NaN,NaN,NaN,None
2,Akumadan,4,PLANTED,1,FORM GHANA,ASUBIMA,2009,42.605816,42.605816,0.00,42.605816,NaN,NaN,NaN,NaN,NaN,None
3,Akumadan,5,PLANTED,1,FORM GHANA,ASUBIMA,2009,63.458447,62.918447,0.54,62.918447,NaN,NaN,NaN,NaN,0.54,None
4,Akumadan,6,PLANTED,1,FORM GHANA,ASUBIMA,2009,78.999204,78.999204,0.00,78.999204,NaN,NaN,NaN,NaN,NaN,None


In [4]:
blk_df = blk_register[['PLANTATION', 'BLOCK', 'RESERVE', 'YEAR PLANTED', 'PRODUCTIVE', 'TEAK']]

In [5]:
blk_df.head()

,PLANTATION,BLOCK,RESERVE,YEAR PLANTED,PRODUCTIVE,TEAK
0,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644
1,Akumadan,2B,ASUBIMA,2010,4.523337,4.523337
2,Akumadan,4,ASUBIMA,2009,42.605816,42.605816
3,Akumadan,5,ASUBIMA,2009,62.918447,62.918447
4,Akumadan,6,ASUBIMA,2009,78.999204,78.999204


In [6]:
blk_df = blk_df.rename(columns={'PLANTATION': 'Plantations', 'BLOCK': 'Block', 'RESERVE': 'Reserve', 'YEAR PLANTED': 'year_planted', 'PRODUCTIVE': 'Area_prod', 'TEAK': 'Area_teak'})

In [7]:
blk_df.head()

,Plantations,Block,Reserve,year_planted,Area_prod,Area_teak
0,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644
1,Akumadan,2B,ASUBIMA,2010,4.523337,4.523337
2,Akumadan,4,ASUBIMA,2009,42.605816,42.605816
3,Akumadan,5,ASUBIMA,2009,62.918447,62.918447
4,Akumadan,6,ASUBIMA,2009,78.999204,78.999204


In [8]:
plt_blk_query = r"SELECT * FROM [Plot-block relation] WHERE AreaType = 'Teak'"

plt_blk = pd.read_sql(plt_blk_query, conn)

plt_blk.head()

C:\Users\GILBERT FG\AppData\Local\Temp\ipykernel_14700\2646592545.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  plt_blk = pd.read_sql(plt_blk_query, conn)


,RelationID,Plantations,Plot no,AreaType,Deviating plant year,Block,Start date,End date
0,74,Tain II,1370,Teak,NaN,PF80,2020-05-01,9999-12-31 00:00:00
1,76,Tain II,1372,Teak,NaN,C17,2020-05-01,9999-12-31 00:00:00
2,77,Tain II,1366,Teak,NaN,C17,2020-05-01,9999-12-31 00:00:00
3,78,Tain II,1363,Teak,NaN,C17,2020-05-01,9999-12-31 00:00:00
4,79,Tain II,1374,Teak,NaN,C17,2020-05-01,9999-12-31 00:00:00


In [9]:
plt_blk_df = plt_blk[['Plantations', 'Plot no', 'AreaType', 'Block']]
plt_blk_df.head()

,Plantations,Plot no,AreaType,Block
0,Tain II,1370,Teak,PF80
1,Tain II,1372,Teak,C17
2,Tain II,1366,Teak,C17
3,Tain II,1363,Teak,C17
4,Tain II,1374,Teak,C17


In [10]:
# merge block with plot-block relation
df = pd.merge(blk_df, plt_blk_df, on=['Plantations', 'Block'])
df.head()

,Plantations,Block,Reserve,year_planted,Area_prod,Area_teak,Plot no,AreaType
0,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak
1,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,666,Teak
2,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,4,Teak
3,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,5,Teak
4,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,8,Teak


In [11]:
df = df.rename(columns={'Plot no': 'plot_id'})
df.head()

,Plantations,Block,Reserve,year_planted,Area_prod,Area_teak,plot_id,AreaType
0,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak
1,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,666,Teak
2,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,4,Teak
3,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,5,Teak
4,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,8,Teak


In [12]:
tree_query = r"SELECT * FROM TreeData WHERE AreaType = 'Teak'"

tree_data = pd.read_sql(tree_query, conn)
tree_data.head()

C:\Users\GILBERT FG\AppData\Local\Temp\ipykernel_14700\616845433.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tree_data = pd.read_sql(tree_query, conn)


,Plantations,AreaType,Monitoring year,Monitoring month,Monitoring day,PLOT,TREE NR,Tree SPECIES,Species scientific name,H (m),DBH (cm),Merchantable height (m),REMARKS,Incorrect DBH,Incorrect H,Incorrect H / DBH,Exclude,Justification for exclusion
0,Tain II,Teak,2018.0,1.0,13,1,71.0,Teak,Tectona grandis,7.25,12.0,None,Fire scars,False,False,None,False,None
1,Tain II,Teak,2018.0,1.0,13,1,76.0,Teak,Tectona grandis,8.25,12.0,None,Fire scars,False,False,None,False,None
2,Tain II,Teak,2018.0,1.0,13,1,68.0,Teak,Tectona grandis,7.00,11.8,None,Fire scars,False,False,None,False,None
3,Tain II,Teak,2018.0,1.0,13,1,53.0,Teak,Tectona grandis,7.25,11.5,None,Fire scars,False,False,None,False,None
4,Tain II,Teak,2018.0,1.0,13,1,58.0,Teak,Tectona grandis,7.00,10.5,None,Fire scars,False,False,None,False,None


In [13]:
tree_df = tree_data[['Plantations', 'AreaType', 'Monitoring year', 'Monitoring month', 'Monitoring day', 'PLOT', 'TREE NR', 'Tree SPECIES', 'H (m)', 'DBH (cm)', 'REMARKS']]
tree_df.head()

,Plantations,AreaType,Monitoring year,Monitoring month,Monitoring day,PLOT,TREE NR,Tree SPECIES,H (m),DBH (cm),REMARKS
0,Tain II,Teak,2018.0,1.0,13,1,71.0,Teak,7.25,12.0,Fire scars
1,Tain II,Teak,2018.0,1.0,13,1,76.0,Teak,8.25,12.0,Fire scars
2,Tain II,Teak,2018.0,1.0,13,1,68.0,Teak,7.00,11.8,Fire scars
3,Tain II,Teak,2018.0,1.0,13,1,53.0,Teak,7.25,11.5,Fire scars
4,Tain II,Teak,2018.0,1.0,13,1,58.0,Teak,7.00,10.5,Fire scars


In [14]:
tree_df = tree_df.rename(columns={'PLOT': 'plot_id', 'TREE NR': 'tree_id', 'Tree SPECIES': 'species', 'H (m)': 'height_m', 'DBH (cm)': 'dbh_cm' })
tree_df.head()

,Plantations,AreaType,Monitoring year,Monitoring month,Monitoring day,plot_id,tree_id,species,height_m,dbh_cm,REMARKS
0,Tain II,Teak,2018.0,1.0,13,1,71.0,Teak,7.25,12.0,Fire scars
1,Tain II,Teak,2018.0,1.0,13,1,76.0,Teak,8.25,12.0,Fire scars
2,Tain II,Teak,2018.0,1.0,13,1,68.0,Teak,7.00,11.8,Fire scars
3,Tain II,Teak,2018.0,1.0,13,1,53.0,Teak,7.25,11.5,Fire scars
4,Tain II,Teak,2018.0,1.0,13,1,58.0,Teak,7.00,10.5,Fire scars


In [15]:
# merge tree data to dataframe
df = pd.merge(df, tree_df, on=['Plantations', 'AreaType', 'plot_id'])
df.head()

,Plantations,Block,Reserve,year_planted,Area_prod,Area_teak,plot_id,AreaType,Monitoring year,Monitoring month,Monitoring day,tree_id,species,height_m,dbh_cm,REMARKS
0,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,1.0,Teak,16.50,16.5,None
1,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,2.0,Teak,16.50,21.3,None
2,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,3.0,Teak,17.00,21.7,None
3,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,4.0,Teak,16.25,19.8,None
4,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,5.0,Teak,14.00,15.3,None


In [16]:
plt_hist_query = r"SELECT * FROM [Plot monitoring history] WHERE AreaType = 'Teak'"

plt_hist = pd.read_sql(plt_hist_query, conn)

plt_hist.head()

C:\Users\GILBERT FG\AppData\Local\Temp\ipykernel_14700\5572937.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  plt_hist = pd.read_sql(plt_hist_query, conn)


,MonitoringID,Plantations,Plot no,AreaType,Monitoring year,Monitoring Month,Monitoring day,Default plot size,Plot type,Monitoring team,Shrub cover (%),Herb cover (%),Invasive species cover (%),Erosion,Remarks,Excluded,Justification for exclusion,MDate
0,1,Akumadan,194,Teak,2020,2,24,800.0,Circular,"Samuel,John,Comfort & Miriam",NaN,NaN,NaN,False,None,False,None,2020-02-24
1,2,Akumadan,195,Teak,2020,2,25,800.0,Circular,"Alex,Samuel,John,Comfort & Miriam",NaN,NaN,NaN,False,None,False,None,2020-02-25
2,3,Akumadan,196,Teak,2020,2,25,800.0,Circular,"Alex,Samuel,John,Comfort & Miriam",NaN,NaN,NaN,False,None,False,None,2020-02-25
3,4,Akumadan,197,Teak,2020,2,28,800.0,Circular,"Alex,Samuel,John,Miriam & Comfort",NaN,NaN,NaN,False,None,False,None,2020-02-28
4,5,Akumadan,198,Teak,2020,2,28,800.0,Circular,"Alex,Samuel,John,Comfort & Miriam",NaN,NaN,NaN,False,None,False,None,2020-02-28


In [17]:
plt_hist_df = plt_hist[['Plantations', 'AreaType', 'Plot no', 'Default plot size']]
plt_hist_df.head()

,Plantations,AreaType,Plot no,Default plot size
0,Akumadan,Teak,194,800.0
1,Akumadan,Teak,195,800.0
2,Akumadan,Teak,196,800.0
3,Akumadan,Teak,197,800.0
4,Akumadan,Teak,198,800.0


In [18]:
plt_hist_df = plt_hist_df.rename(columns={'Plot no': 'plot_id', 'Default plot size': 'plot_size_m2'})
plt_hist_df.head()

,Plantations,AreaType,plot_id,plot_size_m2
0,Akumadan,Teak,194,800.0
1,Akumadan,Teak,195,800.0
2,Akumadan,Teak,196,800.0
3,Akumadan,Teak,197,800.0
4,Akumadan,Teak,198,800.0


In [19]:
# merge plot monitoring history to the dataframe
df = pd.merge(df, plt_hist_df, on=['Plantations', 'AreaType', 'plot_id'])
df.head()

,Plantations,Block,Reserve,year_planted,Area_prod,Area_teak,plot_id,AreaType,Monitoring year,Monitoring month,Monitoring day,tree_id,species,height_m,dbh_cm,REMARKS,plot_size_m2
0,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,1.0,Teak,16.5,16.5,None,800.0
1,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,1.0,Teak,16.5,16.5,None,800.0
2,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,2.0,Teak,16.5,21.3,None,800.0
3,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,2.0,Teak,16.5,21.3,None,800.0
4,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,3.0,Teak,17.0,21.7,None,800.0


In [20]:
thin_hist_query = r"SELECT * FROM [Thinning history] WHERE AreaType = 'Teak'"

thin_hist = pd.read_sql(thin_hist_query, conn)
thin_hist.head()

C:\Users\GILBERT FG\AppData\Local\Temp\ipykernel_14700\3473780868.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  thin_hist = pd.read_sql(thin_hist_query, conn)


,ID,Plantation,Block,AreaType,Year thinning,Target density,Final density,Thinning number,Commercial volume,Remarks
0,1,Akumadan,1A,Teak,2010.0,600.0,None,T1,0.00,felled to waste
1,2,Akumadan,1A,Teak,2012.0,450.0,None,T2,NaN,Info not available
2,3,Akumadan,1A,Teak,2015.0,250.0,None,T3,NaN,Info not available
3,4,Akumadan,1A-TH,Teak,2022.0,160.0,None,T4,283.38,None
4,5,Akumadan,20A,Teak,2014.0,450.0,None,T2,NaN,Info not available


In [21]:
thin_hist = thin_hist.rename(columns={'Plantation': 'Plantations'})
thin_hist.head()

,ID,Plantations,Block,AreaType,Year thinning,Target density,Final density,Thinning number,Commercial volume,Remarks
0,1,Akumadan,1A,Teak,2010.0,600.0,None,T1,0.00,felled to waste
1,2,Akumadan,1A,Teak,2012.0,450.0,None,T2,NaN,Info not available
2,3,Akumadan,1A,Teak,2015.0,250.0,None,T3,NaN,Info not available
3,4,Akumadan,1A-TH,Teak,2022.0,160.0,None,T4,283.38,None
4,5,Akumadan,20A,Teak,2014.0,450.0,None,T2,NaN,Info not available


In [22]:
thin_hist_df = thin_hist[['Plantations', 'Block', 'AreaType', 'Year thinning', 'Thinning number']]
thin_hist_df.head()

,Plantations,Block,AreaType,Year thinning,Thinning number
0,Akumadan,1A,Teak,2010.0,T1
1,Akumadan,1A,Teak,2012.0,T2
2,Akumadan,1A,Teak,2015.0,T3
3,Akumadan,1A-TH,Teak,2022.0,T4
4,Akumadan,20A,Teak,2014.0,T2


In [23]:
# merge thining history to the dataframe
df = pd.merge(df, thin_hist_df, on=['Plantations', 'Block', 'AreaType'])

## GRAND DATAFRAME

In [24]:
df.head()

,Plantations,Block,Reserve,year_planted,Area_prod,Area_teak,plot_id,AreaType,Monitoring year,Monitoring month,Monitoring day,tree_id,species,height_m,dbh_cm,REMARKS,plot_size_m2,Year thinning,Thinning number
0,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,1.0,Teak,16.5,16.5,None,800.0,2013.0,T1
1,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,1.0,Teak,16.5,16.5,None,800.0,2018.0,T2
2,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,1.0,Teak,16.5,16.5,None,800.0,2022.0,T3
3,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,1.0,Teak,16.5,16.5,None,800.0,2013.0,T1
4,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,1.0,Teak,16.5,16.5,None,800.0,2018.0,T2


## DATA CLEANSING AND TRANSFORMATION

In [25]:
df.shape

(4229303, 19)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4229303 entries, 0 to 4229302
Data columns (total 19 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Plantations       object 
 1   Block             object 
 2   Reserve           object 
 3   year_planted      int64  
 4   Area_prod         float64
 5   Area_teak         float64
 6   plot_id           object 
 7   AreaType          object 
 8   Monitoring year   float64
 9   Monitoring month  float64
 10  Monitoring day    int64  
 11  tree_id           float64
 12  species           object 
 13  height_m          float64
 14  dbh_cm            float64
 15  REMARKS           object 
 16  plot_size_m2      float64
 17  Year thinning     float64
 18  Thinning number   object 
dtypes: float64(9), int64(2), object(8)
memory usage: 613.1+ MB


In [27]:
df.isna().sum()

Plantations               0
Block                     0
Reserve                   0
year_planted              0
Area_prod                 0
Area_teak                 0
plot_id                   0
AreaType                  0
Monitoring year           0
Monitoring month          0
Monitoring day            0
tree_id                  54
species                 995
height_m               2375
dbh_cm                 6483
REMARKS             2842071
plot_size_m2              0
Year thinning             0
Thinning number           0
dtype: int64

In [28]:
df = df.dropna(subset=['height_m', 'dbh_cm'])

In [29]:
df.isna().sum()

Plantations               0
Block                     0
Reserve                   0
year_planted              0
Area_prod                 0
Area_teak                 0
plot_id                   0
AreaType                  0
Monitoring year           0
Monitoring month          0
Monitoring day            0
tree_id                   0
species                 947
height_m                  0
dbh_cm                    0
REMARKS             2837816
plot_size_m2              0
Year thinning             0
Thinning number           0
dtype: int64

In [30]:
df.shape

(4222726, 19)

In [31]:
df['species'].unique()

array(['Teak', 'Akonkodie', 'Watapuo', 'Akuakuo-Ninsuo',
       'Unknown indigenous', 'Okoro', 'Onyina', 'Foto', 'Konkroma',
       'Albizia', 'Mahogany', 'Khaya ivorensis', 'Odum', 'Nyankyerene',
       'Trilepisium madagascariense', 'Dead', 'Akye', 'onyono',
       'Awiemfosamina', 'kotrenfro', 'Ofram', 'Kyenkyen', 'Cedrela', None,
       'Cassia', 'Acacia', 'Acasia', 'Wawa', 'Asanfena', 'Kane',
       'Albizia adianthifolia', 'Sofo', 'Hontan',
       'Tetrapleura tetraptera', 'prekese', 'Prekese', 'Afzelia',
       'kwakuanisuo', 'coconisuol', 'Kookonisuo', 'Mango', 'Wonton',
       'Nofotene', 'Kigelia africana', 'Hyedua', 'Nyamedua', 'Otie',
       'Kotreamfo', 'Womba', 'Alstonia boonei', 'Anacardium occidentale',
       'Wama', 'Kusia', 'Funtumia africana', 'Okure', 'York'],
      dtype=object)

In [34]:
unwanted_list = set(df['species'])
teak = 'Teak'

for i in unwanted_list:
    if i != teak:
        df = df[~((df['species'] == i))]


In [35]:
df['species'].unique()

array(['Teak', None], dtype=object)

In [36]:
df.shape

(4196018, 19)

In [37]:
df['REMARKS'].unique()

array([None, '', 'Wildling', 'Coppice',
       'Diameter taken with diameter tape - Wildling - marked',
       'Diameter taken with diameter tape',
       'Diameter taken with diameter tape - marked',
       'Outside planting line - Wildling', 'Dead', 'Top dead',
       'Broken top',
       'diseased - dead tree - Diameter taken with diameter tape',
       'diseased - Diameter taken with diameter tape',
       'Diameter taken with diameter tape - Wildling',
       'Diameter taken with diameter tape - Coppice',
       'Diameter taken with diameter tape - Coppice - marked',
       'Beside a termite mould', 'Dying', 'marked',
       'leaves eaten by grasshoppers', 'Top broken - Wildling',
       'Diameter taken with diameter tape - Coppice - dead tree',
       'Diameter taken with diameter tape - dead tree',
       'Diameter taken with diameter tape - Wildling - dead tree',
       'half dead', 'Double stem', 'Tape', 'Tape - marked', 'Top broken',
       'Dead,Top broken', 'Marked', 'dead/

In [39]:
import re

pattern = r"(Wildling|Coppice|Broken top|diseased|dead|Dead|DEAD)"

remarks = set(df['REMARKS'])

for i in remarks:
    if i is not None and re.search(pattern, i):
        df = df[~((df['REMARKS'] == i))]

In [40]:
df['REMARKS'].unique()

array([None, '', 'Diameter taken with diameter tape',
       'Diameter taken with diameter tape - marked',
       'Beside a termite mould', 'Dying', 'marked',
       'leaves eaten by grasshoppers', 'Double stem', 'Tape',
       'Tape - marked', 'Top broken', 'Marked',
       'Diameter < 0. Diameter taken with diameter tape', 'top broken',
       'towards bumgate 2', 'coppice, marked',
       'Diameter taken with caliper',
       'Strange H/D. DBH assumed to be 15,2 instead of 5,2',
       'Teak dieback', 'Old teak', '2 stems',
       'Abnormal tree,correct measurements',
       'Black ant attack from base to top',
       'this could be an old teak tree within the block,its same on the field sheet',
       'D above 1.3. Diameter taken with diameter tape',
       'D above 1.3. Diameter taken diameter tape',
       'Top broken. Diameter taken with diameter tape',
       "Remark in field sheet is 1. Assumed to be 'marked' - marked",
       'wildling', 'coppice',
       'DBH missing on fiel

In [41]:
df.shape

(4102741, 19)

In [42]:
pattern = r"(Top broken|coppice|wildling|fork|Beetle|Old)"

remarks = set(df['REMARKS'])

for i in remarks:
    if i is not None and re.search(pattern, i, re.IGNORECASE):
        df = df[~((df['REMARKS'] == i))]

In [43]:
df.shape

(4087245, 19)

In [44]:
df['REMARKS'].unique()

array([None, '', 'Diameter taken with diameter tape',
       'Diameter taken with diameter tape - marked',
       'Beside a termite mould', 'Dying', 'marked',
       'leaves eaten by grasshoppers', 'Double stem', 'Tape',
       'Tape - marked', 'Marked',
       'Diameter < 0. Diameter taken with diameter tape',
       'towards bumgate 2', 'Diameter taken with caliper',
       'Strange H/D. DBH assumed to be 15,2 instead of 5,2',
       'Teak dieback', '2 stems', 'Abnormal tree,correct measurements',
       'Black ant attack from base to top',
       'D above 1.3. Diameter taken with diameter tape',
       'D above 1.3. Diameter taken diameter tape',
       "Remark in field sheet is 1. Assumed to be 'marked' - marked",
       'DBH missing on field sheet. Assumed to be 12.6 corresponding the other trees in the plot',
       'DBH incorrect on field sheet. Was 4, but assumed to be 14',
       'Broken', 'Strange H/D. DBH assumed to be 14.9 instead of 4.9',
       'Teak dieback - marked',
  

In [48]:
remarks = set(df['REMARKS'])

pattern = r"(dieback|Diseased|Broken|Abnormal|crooked|marked)"

for i in remarks:
    if i is not None and re.search(pattern, i, re.IGNORECASE):
        df = df[~((df['REMARKS'] == i))]

In [49]:
df.shape

(3908949, 19)

In [50]:
df['REMARKS'].unique()

array([None, '', 'Diameter taken with diameter tape',
       'Beside a termite mould', 'Dying', 'leaves eaten by grasshoppers',
       'Double stem', 'Tape',
       'Diameter < 0. Diameter taken with diameter tape',
       'towards bumgate 2', 'Diameter taken with caliper',
       'Strange H/D. DBH assumed to be 15,2 instead of 5,2', '2 stems',
       'Black ant attack from base to top',
       'D above 1.3. Diameter taken with diameter tape',
       'D above 1.3. Diameter taken diameter tape',
       'DBH missing on field sheet. Assumed to be 12.6 corresponding the other trees in the plot',
       'DBH incorrect on field sheet. Was 4, but assumed to be 14',
       'Strange H/D. DBH assumed to be 14.9 instead of 4.9',
       'Dying. Diameter taken with caliper', 'Beaten-up', 'Hung up',
       'Diameter taken with diameter tape; monitoring day unknown',
       '3 stems', 'Beaten-up. Top slashed',
       'One tree on  the boundary not actually in the plot',
       '2 trees outside plot f

In [51]:
df.isna().sum()

Plantations               0
Block                     0
Reserve                   0
year_planted              0
Area_prod                 0
Area_teak                 0
plot_id                   0
AreaType                  0
Monitoring year           0
Monitoring month          0
Monitoring day            0
tree_id                   0
species                 944
height_m                  0
dbh_cm                    0
REMARKS             2827365
plot_size_m2              0
Year thinning             0
Thinning number           0
dtype: int64

## DATA ANALYSIS

#### BASIC STATISTICS

##### 1. Nha - Stand density (#trees/ha)

In [52]:
df.head()

,Plantations,Block,Reserve,year_planted,Area_prod,Area_teak,plot_id,AreaType,Monitoring year,Monitoring month,Monitoring day,tree_id,species,height_m,dbh_cm,REMARKS,plot_size_m2,Year thinning,Thinning number
0,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,1.0,Teak,16.5,16.5,None,800.0,2013.0,T1
1,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,1.0,Teak,16.5,16.5,None,800.0,2018.0,T2
2,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,1.0,Teak,16.5,16.5,None,800.0,2022.0,T3
3,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,1.0,Teak,16.5,16.5,None,800.0,2013.0,T1
4,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,1.0,Teak,16.5,16.5,None,800.0,2018.0,T2


In [53]:
df['age'] = df['Monitoring year'] - df['year_planted']
df.head()

,Plantations,Block,Reserve,year_planted,Area_prod,Area_teak,plot_id,AreaType,Monitoring year,Monitoring month,Monitoring day,tree_id,species,height_m,dbh_cm,REMARKS,plot_size_m2,Year thinning,Thinning number,age
0,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,1.0,Teak,16.5,16.5,None,800.0,2013.0,T1,13.0
1,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,1.0,Teak,16.5,16.5,None,800.0,2018.0,T2,13.0
2,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,1.0,Teak,16.5,16.5,None,800.0,2022.0,T3,13.0
3,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,1.0,Teak,16.5,16.5,None,800.0,2013.0,T1,13.0
4,Akumadan,2A,ASUBIMA,2008,77.853644,77.853644,667,Teak,2021.0,4.0,13,1.0,Teak,16.5,16.5,None,800.0,2018.0,T2,13.0


In [112]:
# remove trees with less than 3 years
df = df[~((df['age'] < 3.0))]

In [113]:
sd = df.groupby(['Plantations','year_planted', 'Monitoring year', 'age'])['tree_id'].nunique().reset_index(name="#trees")

In [114]:
sd.head()

,Plantations,year_planted,Monitoring year,age,#trees
0,Akumadan,2001,2009.0,8.0,19
1,Akumadan,2001,2010.0,9.0,22
2,Akumadan,2001,2011.0,10.0,18
3,Akumadan,2001,2012.0,11.0,12
4,Akumadan,2001,2013.0,12.0,40


In [115]:
sd['Nha'] = sd['#trees'] * 10000 / df['plot_size_m2']
sd.head()

,Plantations,year_planted,Monitoring year,age,#trees,Nha
0,Akumadan,2001,2009.0,8.0,19,237.5
1,Akumadan,2001,2010.0,9.0,22,275.0
2,Akumadan,2001,2011.0,10.0,18,225.0
3,Akumadan,2001,2012.0,11.0,12,150.0
4,Akumadan,2001,2013.0,12.0,40,500.0
